# Introduction and Business Understanding

More and more electric vehicles can be seen on the streets as they are slowly replacing traditional vehicles with combustion engines. Electric vehicles can do without a fuel tank; instead they have a powerful battery which is used both for moving the vehicle as well as for powering other consumers, such as heating, airconditioning, multimedia, and infotainment. 

Most drivers of electric vehicles are well aware of the "State of Charge" that shows how much power is left in the battery throughout a trip. The fear of a breakdown due to an empty battery has even received its own terminology: "Range Anxiety". 

This Data Science project focusses on the "State of Charge" (SoC) of electric vehicles, explores dependencies to measures taken during a vehicle's trip, and makes predictions of the required battery power for a vehicle's trip. 

For the analysis, a dataset is used which includes 70 trips of a BMW i3 in 2019 in and around Munich, Germany. Throughout the trips a variety of measures have been recorded, such as velocity, acceleration, ambient temperature, elevation, heating, air conditioning, motor torque, and more.

## Structure of this Notebook

- Introduction and Business Understanding
- Importing Required Libraries
- Step 1 | Loading Data
- Step 2 | Data Exploration, Data Cleansing, Data Visualization
- Step 3 | Data Preparation and Saving

# Importing Required Libraries

In [19]:
# Import libraries
import pandas as pd

from os import listdir
from os.path import isfile, join
from os import makedirs

import numpy as np

# Step 1 | Loading Data 

In [3]:
# Retrieve csv filenames from battery_and_heating_data directory
tripdata = []
for f in listdir('battery_and_heating_data'):
    if f[-3:] == 'csv':
        tripdata.append(f)

In [4]:
# Read in all tripdata including the trip's id
trip_df = pd.DataFrame()
for f in tripdata:
    new_df = pd.read_csv(filepath_or_buffer=('battery_and_heating_data/'+f), sep=';', engine='python')
    new_df['trip_id'] = f[:-4]
    
    trip_df = pd.concat([trip_df, new_df], sort=False)

print(trip_df.shape)
trip_df.head()

(1094793, 51)


,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C],Velocity [km/h]]]
0,0.0,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.20,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.1,0.0,574.0,0.0,0.0,0.00,0.0,391.4,-2.21,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.2,0.0,574.0,0.0,0.0,-0.01,0.0,391.4,-2.26,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.3,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.4,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 2 | Data Exploration, Data Cleansing, Data Visualization

To start our data exploration journey, we take the trip_df dataframe and run first queries against it.

In [5]:
number_of_trips = len(trip_df['trip_id'].unique())
print('The dataset contains ', number_of_trips, ' individual trips.')

The dataset contains  70  individual trips.


Upon checking the trips dataset for null values we find:
- (1) Column 'Unnamed: 23' has 99.9999 % missing values. We will remove this column.
- (2) There are two columns for velocity: 'Velocity [km/h]]]' and 'Velocity [km/h]'. We will combine them into one.
- (3) There are two columns for the right vent's temperature: 'Temperature Vent right [°C] ' and 'Temperature Vent right [°C]'. We will investigate further.

In [6]:
(trip_df.isnull().sum() / trip_df.shape[0]).sort_values(ascending=False)

Unnamed: 23                               0.999999
Velocity [km/h]]]                         0.984994
Coolant Volume Flow +500 [l/h]            0.453085
Temperature Defrost central right [°C]    0.431442
Ambient Temperature Sensor [°C]           0.431442
Temperature Coolant Heater Inlet [°C]     0.431442
Temperature Coolant Heater Outlet [°C]    0.431442
Temperature Heat Exchanger Outlet [°C]    0.431442
Temperature Defrost lateral left [°C]     0.431442
Temperature Defrost lateral right [°C]    0.431442
Temperature Defrost central left [°C]     0.431442
Temperature Defrost central [°C]          0.431442
Temperature Footweel Driver [°C]          0.431442
Temperature Footweel Co-Driver [°C]       0.431442
Temperature Feetvent Co-Driver [°C]       0.431442
Temperature Feetvent Driver [°C]          0.431442
Temperature Head Co-Driver [°C]           0.431442
Temperature Head Driver [°C]              0.431442
Temperature Vent right [°C]               0.431442
Temperature Vent central right 

In [7]:
# (1) Removing (almost) empty column 'Unnamed: 23'
trip_df.drop(columns=['Unnamed: 23'], inplace=True)

In [8]:
# (2) Combining two velocity columns into one
## Checking that the two velocity columns do not overlap
print('Number of empty values in column \'Velocity [km/h]\' when \'Velocity [km/h]]]\' is empty: ', 
      trip_df[trip_df['Velocity [km/h]]]'].isna()]['Velocity [km/h]'].isna().sum())
print('Number of empty values in column \'Velocity [km/h]]]\' when \'Velocity [km/h]\' is empty: ', 
      trip_df[trip_df['Velocity [km/h]'].isna()]['Velocity [km/h]]]'].isna().sum())

## Moving content of 'Velocity [km/h]]]' into 'Velocity [km/h]'
trip_df_v = trip_df[trip_df['Velocity [km/h]'].isna()].copy()
trip_df_v['Velocity [km/h]'] = trip_df_v['Velocity [km/h]]]']
trip_df = pd.concat([trip_df[trip_df['Velocity [km/h]]]'].isna()], trip_df_v])

## Removing no longer needed column 'Velocity [km/]]]'
trip_df.drop(columns=['Velocity [km/h]]]'], inplace=True)

Number of empty values in column 'Velocity [km/h]' when 'Velocity [km/h]]]' is empty:  0
Number of empty values in column 'Velocity [km/h]]]' when 'Velocity [km/h]' is empty:  0


In [9]:
# (3) Investigating temperature data from right vent
temp_vent_df = trip_df[['trip_id', 'Temperature Vent right [°C]', 'Temperature Vent right [°C] ']].copy()
temp_vent_df['Difference'] = temp_vent_df['Temperature Vent right [°C]'] - temp_vent_df['Temperature Vent right [°C] ']
print(temp_vent_df['Difference'].describe())

trip_df.rename(columns={'Temperature Vent right [°C]': 'Temperature Vent One [°C]', 
                        'Temperature Vent right [°C] ': 'Temperature Vent Two [°C]'},
              inplace=True)

count    622453.000000
mean         -2.079627
std           1.493197
min          -9.860000
25%          -2.310350
50%          -1.920000
75%          -1.400000
max           3.720000
Name: Difference, dtype: float64


We discover that the values of the two 'Temperature Vent right' columns differ from one another. It can thus be assumed that an error occured during data collection when labeling the data columns. Possibly, one of the columns should have been labeled 'Temperature Vent left [°C]'. To avoid missunderstandings during the upcoming data processing, we decide to remove the vents' locations from the column names and label the columns 'Temperature Vent One [°C]' and 'Temperature Vent Two [°C]' instead.

We are now taking a look at the trips in more details. For that we are visualizing mutliple measures over the time of the trip. The measures we choose to visualize are:
- Velocity
- Acceleration
- Throttle
- Motor Torque
- State of Charge

# Step 3 | Data Preparation

In [10]:
trip_df_by_seconds = trip_df.copy()
trip_df_by_seconds['Time [s]'] = np.floor(trip_df_by_seconds['Time [s]'])
trip_df_by_seconds = trip_df_by_seconds.groupby(['trip_id', 'Time [s]']).mean()
trip_df_by_seconds = trip_df_by_seconds.reset_index()

In [11]:
trip_df_by_seconds.head()

,trip_id,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],...,Temperature Footweel Driver [°C],Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent Two [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent One [°C]
0,TripA01,0.0,0.000,574.0,0.000,0.050,-0.017,0.0,391.400,-2.291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TripA01,1.0,0.000,574.0,0.000,0.084,-0.013,0.0,391.363,-2.552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TripA01,2.0,0.000,574.0,8.770,7.655,0.036,0.0,391.283,-3.405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TripA01,3.0,0.869,574.0,16.683,45.279,0.404,0.0,391.124,-5.232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TripA01,4.0,2.251,574.0,13.427,25.005,0.162,0.0,391.196,-4.560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
## Save aggregated trips to csv
makedirs('battery_and_heating_data/aggregated_trips', exist_ok=True)
trip_df_by_seconds.to_csv('battery_and_heating_data/aggregated_trips/trips_df_by_seconds.csv', sep=';')

# What Remains to be Said

The data used within this Notebook has been downloaded from Kaggle (https://www.kaggle.com/atechnohazard/battery-and-heating-data-in-real-driving-cycles). It has originally been published by Matthias Steinstraeter (Technical University of Munich, Institute of Automotive Technology), Johannes Buberger (Technical University of Munich, Institute of Automotive Technology), and Dimitar Trifonov (Technical University of Munich, Institute of Automotive Technology) in IEEE Data Portal (https://ieee-dataport.org/open-access/battery-and-heating-data-real-driving-cycles).

Author of this Notebook is: F B

Created in March 2022